In [12]:
import cv2
import numpy as np

# Step 1: Load the black-and-white (grayscale) image
image_path = r"C:\Users\himub\OneDrive\Desktop\python\All_Image\black_white.jpeg"  # Change to your image file
gray_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if gray_image is None:
    print("Error: Image not found.")
    exit()

# Step 2: Apply a color map (Try different colormaps)
colorized_image = cv2.applyColorMap(gray_image, cv2.COLORMAP_JET)  # Change to COLORMAP_RAINBOW or others

# Step 3: Show the original and colorized images
cv2.imshow("Black & White Image", gray_image)
cv2.imshow("Colorized Image", colorized_image)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [11]:
import cv2
import numpy as np

# Load pre-trained model
proto_file = "colorization_deploy_v2.prototxt"
model_file = "colorization_release_v2.caffemodel"
points_file = "pts_in_hull.npy"

# Load the model
net = cv2.dnn.readNetFromCaffe(proto_file, model_file)
pts_in_hull = np.load(points_file)

# Load cluster centers as a 1x1 convolution kernel
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
net.getLayer(class8).blobs = [pts_in_hull.reshape(2, 313, 1, 1)]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, np.float32)]

def colorize_image(image_path, output_path="colorized_output.jpg"):
    # Load the grayscale image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Convert grayscale to 3-channel RGB (needed for LAB conversion)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    # Convert to LAB color space
    lab_img = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    L = lab_img[:, :, 0]  # Extract lightness channel

    # Resize the lightness channel to 224x224 for model input
    L_input = cv2.resize(L, (224, 224))
    L_input = L_input.astype("float32") / 255.0
    L_input = L_input - 0.5
    L_input = L_input * 2.0
    L_input = cv2.dnn.blobFromImage(L_input)

    # Pass through the network
    net.setInput(L_input)
    ab_output = net.forward()[0, :, :, :].transpose((1, 2, 0))

    # Resize ab channels back to original image size
    ab_output = cv2.resize(ab_output, (img.shape[1], img.shape[0]))

    # Merge with original lightness channel
    lab_colorized = np.concatenate((L[:, :, np.newaxis], ab_output), axis=2)
    colorized_img = cv2.cvtColor(lab_colorized, cv2.COLOR_LAB2BGR)
    colorized_img = np.clip(colorized_img, 0, 255).astype("uint8")

    # Save and display the result
    cv2.imwrite(output_path, colorized_img)
    print(f"Colorized image saved as {output_path}")

    # Show the images
    cv2.imshow("Original (Grayscale)", cv2.imread(image_path))
    cv2.imshow("Colorized", colorized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
colorize_image("black_and_white.jpg", "colorized_output.jpg")


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\caffe\caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "colorization_deploy_v2.prototxt" in function 'cv::dnn::ReadProtoFromTextFile'


In [8]:
import os
print(os.path.exists("colorization_deploy_v2.prototxt"))  # Should print True
print(os.path.exists("colorization_release_v2.caffemodel"))  # Should print True
print(os.path.exists("pts_in_hull.npy"))  # Should print True


False
False
False
